# Getting Started with RAG in DSPy

This notebook will show you how to use DSPy to compile a RAG program! DSPy compilation is a fairly new tool for LLM developers, so let's start with an overview of the concept. By `compiling`, we mean finding the prompts that elicit the behavior we want from LLMs when connected in some kind of pipeline.

For example, RAG is a very common LLM pipeline. In it's simplest form, RAG consists of 2 steps, (1) Retrieve and (2) Answer a Question. Part (2), Answering a Question, has an associated prompt, for example, people generally use:

```
--

Please answer the question based on the following context.

context  {context}

question {question}

--
```

This prompt may be a good initial point for an LLM to understand the task. However, it is not the *optimal* prompt. DSPy optimizes the prompt for you by jointly (1) tweaking the instructions, such as rewriting an initial prompt like: 

```
Please answer the question based on the following context.
```

to 

```
Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.
```

Further, DSPy (2) finds examples of desired input-outputs in the prompt to further improve performance, also known as `In-Context Learning`. In this example, we will begin with the simple prompt: `Please answer the question based on the following context.` and end up with:

```

```

In order to leverage black-box optimization techniques like random search, bayesian optimization, or evolutionary algorithms, we need a metric. Coming up with metrics to describe desired system behavior has been a longstanding challenge in Machine Learning research. Excitingly, LLMs have made amazing progress. For example, we can evaluate a RAG answer by prompting an LLM with, `Is the assessed text grounded in the context? Say no if it includes significant facts not in the context`. We then optimize the RAG program to increase the metric LLM's assessment of answer quality.

This example contains 4 parts:

- 0: DSPy Settings and Installation
- 1: DSPy Datasets with `dspy.Example`
- 2: LLM Metrics in DSPy
- 3: LLM Programming with `dspy.Module`
- 4: Optimization with `BootstrapFewShot`, `BootstrapFewShotRandomSearch`, and `BayesianSignatureOptimizer`.


We are using 2 datasets for this example. Firstly, we have an index of the Weaviate Blog Posts. We will use the Weaviate Blog Posts as the retrieved context to help with our second dataset, the Weaviate FAQs. The Weaviate FAQs consists of 44 question-answer pairs of frequently asked Weaviate questions such as: `Do I need to know about Docker (Compose) to use Weaviate?`

We isolate 10 examples to use as our test set and optimize our program with the remaining 34.

Our uncompiled RAG program achieves a score of 270 on the held-out test set.

Our RAG program compiled with the `BayesianSignatureOptimizer` achieves a score of 340! A ~30% improvement!

# 0: DSPy Settings and Installations

In [ ]:
!pip install dspy-ai==2.1.9 weaviate-client==3.26.2 > /dev/null

In [1]:
import openai
openai.api_key = "sk-foobar"

In [4]:
# Connect to Weaviate Retriever and configure LLM
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate
import openai


llm = dspy.OpenAI(model="gpt-3.5-turbo")

# ollamaLLM = dspy.OpenAI(api_base="http://localhost:11434/v1/", api_key="ollama", model="mistral-7b-instruct-v0.2-q6_K", stop='\n\n', model_type='chat')
# Thanks Knox! https://twitter.com/JR_Knox1977/status/1756018720818794916/photo/1

weaviate_client = weaviate.Client("http://localhost:8080")
retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)
# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm, rm=retriever_model)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [6]:
print(dspy.settings.lm("Write a 3 line poem about neural networks."))
context_example = dspy.OpenAI(model="gpt-4")

with dspy.context(llm=context_example):
    print(context_example("Write a 3 line poem about neural networks."))

['Neurons intertwine,\nSynapses ignite, design,\nMinds in code align.']
['In the realm of silicon thought,\nNeural networks, with knowledge fraught.\nA dance of data, endlessly taught.']


# 1. DSPy Datasets with `dspy.Example`

Our retrieval engine is filled with chunks from Weaviate Blog posts.

Please see weaviate/recipes/integrations/dspy/Weaviate-Import.ipynb for a full tutorial.

# Import FAQs from a markdown file

In [7]:
# Load FAQs
import re

f = open("faq.md")
markdown_content = f.read()

def parse_questions(markdown_content):
    # Regular expression pattern for finding questions
    question_pattern = r'#### Q: (.+?)\n'

    # Finding all questions
    questions = re.findall(question_pattern, markdown_content, re.DOTALL)

    return questions

# Parsing the markdown content to get only questions
questions = parse_questions(markdown_content)

# Displaying the first few extracted questions
questions[:5]  # Displaying only the first few for brevity

['Why would I use Weaviate as my vector database?',
 'What is the difference between Weaviate and for example Elasticsearch?',
 'Do you offer Weaviate as a managed service?',
 'How should I configure the size of my instance?',
 'Do I need to know about Docker (Compose) to use Weaviate?']

In [8]:
len(questions)

44

# Wrap each FAQ into an `Example` object

The dspy `Example` object optionally lets you attach metadata, or additional labels, to input/output pairs.

For example, you may want to jointly supervise the answer as well as the context the retrieval system produced to feed into the answer generator.

In [9]:
# Load into dspy datasets
import dspy

# ToDo, add random splitting -- maybe wrap this entire thing in a cross-validation loop
trainset = questions[:20] # 20 examples for training
devset = questions[20:30] # 10 examples for development
testset = questions[30:] # 14 examples for testing

trainset = [dspy.Example(question=question).with_inputs("question") for question in trainset]
devset = [dspy.Example(question=question).with_inputs("question") for question in devset]
testset = [dspy.Example(question=question).with_inputs("question") for question in testset]

In [12]:
devset[0]

Example({'question': 'Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)'}) (input_keys={'question'})

# 2. LLM Metrics

Define a Metric for Performance.

In [17]:
# This is a WIP, the next step is to optimize this metric as itself a DSPy module (pretty meta)

# Reference - https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py

metricLM = dspy.OpenAI(model='gpt-4', max_tokens=1000, model_type='chat')

# Signature for LLM assessments.

class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer
    question = gold.question
    
    print(f"Test Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    
    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"
    
    with dspy.context(lm=metricLM):
        context = dspy.Retrieve(k=5)(question).passages
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)
    
    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    
    
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)
    
    return total / 5.0

## Inspect the metric

In [18]:
test_example = dspy.Example(question="What do cross encoders do?")
test_pred = dspy.Example(answer="They re-rank documents.")

type(llm_metric(test_example, test_pred))

Test Question: What do cross encoders do?
Predicted Answer: predicted_answer
Faithful: 5
Detail: 1
Overall: 5


float

In [19]:
test_example = dspy.Example(question="What do cross encoders do?")
test_pred = dspy.Example(answer="They index data.")

type(llm_metric(test_example, test_pred))

Test Question: What do cross encoders do?
Predicted Answer: predicted_answer
Faithful: 1
Detail: 1
Overall: 1


float

In [20]:
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Assessed Question: The evaluation criterion.

Assessed Answer: The answer to the question.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context: N/A

Assessed Question: Is the assessed answer detailed?

Assessed Answer: They index data.

Reasoning: Let's think step by step in order to produce the assessment answer. We need to consider if the answer provides enough detail to fully answer the question. In this case, the answer "They index data" is very vague and does not provide any specific details about who "they" are or what it means to "index data". 

Assessment Answer: 1







Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Asse

# 3. The DSPy Programming Model

This block of first code will initilaize the `GenerateAnswer` signature.

Then we will compose a `dspy.Module` consisting of:
- Retrieve
- GenerateAnswer

The DSPy programming model is one of the most powerful aspects of DSPy, we get:
- An intuitive interface to compose prompts into programs.
- A clean way to organize prompts into Signatures.
- Structured output parsing with `dspy.OutputField`
- Built-in prompt extensions such as `ChainOfThought`, `ReAct`, and more!

In [23]:
class GenerateAnswer(dspy.Signature):
    """Answer questions based on the context."""
    
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

In [24]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(answer=prediction.answer)

# A little more info on built-in dspy modules

The DSPy programming model gives you a lot of cool features out of the box. Observe how different modules implement signatures with additional prompting techniques like `ChainOfThought` and `ReAct`. `Predict` is the base class to observe what a standrd prompt looks like without the module extensions.

### dspy.Predict

In [30]:
dspy.Predict(GenerateAnswer)(question="What are Cross Encoders?")
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts
Question: ${question}
Answer: ${answer}

---

Question: What are Cross Encoders?
Answer: Context: Cross Encoders are a type of neural network model used in natural language processing tasks. They are designed to encode pairs of sentences and generate a similarity score between them.

Question: How do Cross Encoders work?
Answer: Cross Encoders work by taking two sentences as input and encoding them into fixed-length vectors. These vectors are then compared using a similarity metric to determine the similarity score between the sentences.

Question: What is the purpose of Cross Encoders?
Answer: The purpose of Cross Encoders is to measure the semantic similarity between pairs of sentences. They can be used in various applications such as question answering, text classification, and information retrieval.

Question: How are Cross Encoders different from other neural network 

### dspy.ChainOfThought

In [31]:
dspy.ChainOfThought(GenerateAnswer)(question="What are Cross Encoders?")
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
produce the answer. We will first define what cross encoders are and then explain their purpose.

Answer: Cross Encoders are a type of neural network model that are used in natural language processing tasks. They are designed to encode pairs of sentences or documents and produce a similarity score between them. The purpose of cross encoders is to capture the semantic relationship between two pieces of text and determine how similar or related they are.

Question: What are Cross Encoders?

Reasoning: Let's think step by step in order to produce the answer. We will first define what cross encoders are and then explain their purpose.

Answer: Cross Encoders are a type of neural network model that are used in natural language processing tasks.

### dspy.ReAct

In [32]:
dspy.ReAct(GenerateAnswer, tools=[dspy.settings.rm])(question="What are cross encoders?")
llm.inspect_history(n=1)





You will be given `context`, `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Thought 1: next steps to take based on last observation

Action 1: always either Search[query] or, when done, Finish[answer]

Observation 1: observations based on action

Thought 2: next steps to take based on last observation

Action 2: always either Search[query] or, when done, Finish[answer]

---

Context:
Cross encoders are a type of neural network model used in natural language processing tasks. They are designed to encode pairs of sentences or documents and captu

# Initialize DSPy Program

In [33]:
uncompiled_rag = RAG()

# Test uncompiled inference 

In [34]:
print(uncompiled_rag("What are re-rankers in search engines?").answer)

Re-rankers in search engines are algorithms or models that are used to reorder or re-rank search results based on additional criteria or features. They can be used to improve the relevance and personalization of search results by considering factors such as user preferences, document features, metadata, and context.


# Check the last call to the LLM

In [35]:
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «They offer the advantage of further reasoning about the relevance of results without needing specialized training. Cross Encoders can be interfaced with Weaviate to re-rank search results, trading off performance for slower search speed. * **Metadata Rankers** are context-based re-rankers that use symbolic features to rank relevance. They take into account user and document features, such as age, gender, location, preferences, release year, genre, and box office, to predict the relevance of candidate documents. By incorporating metadata features, these rankers offer a more personalized and context-aware search experience.»
[2] «Taken directly from the paper, “Our findings indicate that cross-encoder re-rankers can efficiently be impro

# 4. DSPy Optimization

# Evaluate our RAG Program before it is compiled

In [38]:
# Reminder our dataset looks like this:

devset[0]

Example({'question': 'Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)'}) (input_keys={'question'})

In [39]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

evaluate(RAG(), metric=llm_metric)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Test Question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted Answer: predicted_answer


Average Metric: 2.0 / 1  (200.0):  10%|█         | 1/10 [00:00<00:04,  2.03it/s]

Faithful: 1
Detail: 3
Overall: 5
Test Question: How can I retrieve the total object count in a class?
Predicted Answer: predicted_answer


Average Metric: 3.4 / 2  (170.0):  20%|██        | 2/10 [00:00<00:03,  2.28it/s]

...

Average Metric: 24.400000000000002 / 10  (244.0): 100%|█| 10/10 [00:06<00:00,  1

Faithful: 1
Detail: 2
Overall: 5
Average Metric: 24.400000000000002 / 10  (244.0%)



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),No specific information is provided in the given context about support for multiple versions of the query/document embedding models to co-exist at a given time.,2.0
1,How can I retrieve the total object count in a class?,"To retrieve the total object count in a class, you can use the ""count"" function provided by the programming language or framework you are using....",1.4
2,How do I get the cosine similarity from Weaviate's certainty?,"Weaviate does not directly provide the cosine similarity from its certainty value. The certainty value in Weaviate represents the confidence level of the result, but...",1.6
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,The quality of search results changes depending on the specified limit because language models are constrained by input length and can only provide a limited...,2.6
4,Why did you use GraphQL instead of SPARQL?,"The given context does not provide any information about the use of GraphQL instead of SPARQL. Therefore, we cannot determine the reason for using GraphQL...",2.0


244.0

# Metric Analysis

The maximum value per rating is (5 + 5*2 + 5) / 5 = 4

4 * 10 test questions = 40

In [40]:
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «There is no performance gain if we have more parallelization than there are available CPUs. However, each thread needs additional memory. So with 32 parallel imports, we had the worst of both worlds: High memory usage and no performance gains beyond 8. With the fix, even if you import from multiple clients, Weaviate automatically handles the parallelization to ensure that it does not exceed the number of CPU cores. As a result, you get the maximum performance without "unnecessary" memory usage. ### HNSW optimization

Next, we optimized memory allocations for the HNSW (vector) index.»
[2] «The past for vector searching definitely was not a “simpler time”, and the appeal of modern vector databases like Weaviate is pretty clear given thi

In [41]:
metricLM.inspect_history(n=3)





Assess the quality of an answer to a question.

---

Follow the following format.

Context: The context for answering the question.

Assessed Question: The evaluation criterion.

Assessed Answer: The answer to the question.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Context: N/A

Assessed Question: Is the assessed answer detailed?

Assessed Answer: No information is provided in the given context about whether Weaviate uses Hnswlib or not.

Reasoning: Let's think step by step in order to produce the assessment answer. We can see that the answer is not detailed. It simply states that there is no information provided in the given context, but it does not provide any additional information or insight into the topic at hand. 

Assessment Answer: 2







Assess the quality of an answer to a question.

---

Follow the following format.

Context: The 

# BootstrapFewShot

In [42]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=llm_metric, max_labeled_demos=8, max_rounds=3)

# also common to init here, e.g. Rag()
compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Test Question: Why would I use Weaviate as my vector database?
Predicted Answer: predicted_answer


  5%|██▏                                         | 1/20 [00:19<06:15, 19.74s/it]

Faithful: 5
Detail: 5
Overall: 5
Test Question: What is the difference between Weaviate and for example Elasticsearch?
Predicted Answer: predicted_answer


 10%|████▍                                       | 2/20 [00:40<06:02, 20.17s/it]

...

Faithful: 1
Detail: 2
Overall: 5
Test Question: Do I need to know about Docker (Compose) to use Weaviate?
Predicted Answer: predicted_answer


 25%|███████████                                 | 5/20 [01:51<05:34, 22.32s/it]


Faithful: 5
Detail: 5
Overall: 5


  0%|                                                    | 0/20 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 2.


### Inspect the compiled prompt

In [48]:
compiled_rag("What do cross encoders do?").answer

'Cross encoders are ranking models used for content-based re-ranking. They take a query and a document as input and output a score indicating the relevance of the document to the query. They can be interfaced with Weaviate to re-rank search results.'

In [49]:
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «Then we will see how the text vectorization process can be tweaked, before wrapping up by discussing a few considerations also. ## Background

I often find myself saying that Weaviate makes it fast and easy to produce a vector database from text. But it can be easy to forget just how fast and how easy it can make things. It is true that even in the “old days” of say, five to ten years ago, producing a database with vector capabilities was technically possible. You *simply* had to (*inhales deeply*) develop a vectorization algorithm, vectorize the data, build a vector index, build a database with the underlying data, integrate the vector index with the database, then forward results from a vector index query to the database and combine

### Evaluate the Compiled RAG Program

In [52]:
evaluate(compiled_rag, metric=llm_metric)



  0%|                                                    | 0/10 [00:00<?, ?it/s]

Test Question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted Answer: predicted_answer




Average Metric: 2.2 / 1  (220.0):   0%|                  | 0/10 [00:00<?, ?it/s]

Average Metric: 2.2 / 1  (220.0):  10%|█         | 1/10 [00:00<00:03,  2.46it/s]

Faithful: 1
Detail: 4
Overall: 5
Test Question: How can I retrieve the total object count in a class?
Predicted Answer: predicted_answer




Average Metric: 5.6 / 2  (280.0):  10%|█         | 1/10 [00:00<00:03,  2.46it/s]

Average Metric: 5.6 / 2  (280.0):  20%|██        | 2/10 [00:00<00:03,  2.01it/s]

...

Faithful: 5
Detail: 5
Overall: 5
Test Question: Does Weaviate use Hnswlib?
Predicted Answer: predicted_answer




Average Metric: 27.400000000000002 / 10  (274.0):  90%|▉| 9/10 [01:50<00:16, 16.

Average Metric: 27.400000000000002 / 10  (274.0): 100%|█| 10/10 [01:50<00:00, 11

Faithful: 4
Detail: 5
Overall: 5
Average Metric: 27.400000000000002 / 10  (274.0%)


,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),The given context does not provide any information about whether there is support for multiple versions of the query/document embedding models to co-exist at a...,2.2
1,How can I retrieve the total object count in a class?,The given context does not provide any information about how to retrieve the total object count in a class.,3.4
2,How do I get the cosine similarity from Weaviate's certainty?,The given context does not provide any information about how to get the cosine similarity from Weaviate's certainty.,1.6
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,The quality of search results can change depending on the specified limit because language models are constrained by input length. They can only provide a...,4.0
4,Why did you use GraphQL instead of SPARQL?,The given context does not provide any information about why GraphQL was used instead of SPARQL.,1.6


274.0

# BootstrapFewShotWithRandomSearch

In [ ]:
# Accidentally spent $12 on this with `num_candidate_programs=20`, caution!

In [58]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(metric=llm_metric, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=2)

# also common to init here, e.g. Rag()
second_compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 2 candidate sets.




  0%|                                                    | 0/20 [00:00<?, ?it/s]

Test Question: Why would I use Weaviate as my vector database?
Predicted Answer: predicted_answer
Test Question: What is the difference between Weaviate and for example Elasticsearch?
Predicted Answer: predicted_answer




Average Metric: 0.0 / 1  (0.0):   0%|                    | 0/20 [00:00<?, ?it/s]

Average Metric: 0.0 / 1  (0.0):   5%|▌           | 1/20 [00:00<00:08,  2.20it/s]

Average Metric: 4.0 / 2  (200.0):   5%|▌         | 1/20 [00:00<00:08,  2.20it/s]

Average Metric: 4.0 / 2  (200.0):  10%|█         | 2/20 [00:00<00:04,  3.98it/s]

Faithful: No
Detail: 1
Overall: 5
Error for example in dev set: 		 could not convert string to float: 'No'
Faithful: 5
Detail: 5
Overall: 5
Test Question: How should I configure the size of my instance?
Predicted Answer: predicted_answer
Test Question: Do you offer Weaviate as a managed service?
Predicted Answer: predicted_answer


...

Average Metric: 42.2 / 19  (222.1):  95%|██████▋| 19/20 [03:02<00:09,  9.44s/it]

Faithful: 5
Detail: 5
Overall: 3
Test Question: How do you deal with words that have multiple meanings?
Predicted Answer: predicted_answer


Average Metric: 46.2 / 20  (231.0): 100%|███████| 20/20 [03:34<00:00, 10.74s/it]

Faithful: 5
Detail: 5
Overall: 5
Average Metric: 46.2 / 20  (231.0%)
Score: 231.0 for set: [4]
Scores so far: [243.0, 243.0, 260.0, 288.0, 231.0]
Best score: 288.0
Average of max per entry across top 1 scores: 2.8800000000000003
Average of max per entry across top 2 scores: 3.06
Average of max per entry across top 3 scores: 3.11
Average of max per entry across top 5 scores: 3.1399999999999997
Average of max per entry across top 8 scores: 3.1399999999999997
Average of max per entry across top 9999 scores: 3.1399999999999997
5 candidate programs found.



sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x289ca6e60>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x289ca48e0>


In [59]:
second_compiled_rag("What do cross encoders do?")

Prediction(
    answer='Cross encoders are ranking models used for content-based re-ranking. They are used to give a score indicating the relevance of a document to a query.'
)

In [60]:
llm.inspect_history(n=1)





Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «The schema is the place to define, among other things, the data type and vectorizer to be used, as well as cross-references between classes. As a corollary, the vectorization process can be modified for each class by setting the relevant schema options. In fact, you can [define the data schema](/developers/weaviate/manage-data/collections) for each class individually. All this means that you can also use the schema to tweak Weaviate's vectorization behavior. The relevant variables for vectorization are `dataType` and those listed under `moduleConfig` at both the class level and property level.»
[2] «![Hacktober video](img/hacktober.gif)

### [Weaviate Academy](/developers/academy) & [Workshops](/learn/workshops)
Weaviate Academy and W

In [61]:
evaluate(second_compiled_rag, metric=llm_metric)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Test Question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted Answer: predicted_answer


Average Metric: 3.6 / 1  (360.0):  10%|█         | 1/10 [00:21<03:12, 21.35s/it]

Faithful: 5
Detail: 3
Overall: 5
Test Question: How can I retrieve the total object count in a class?
Predicted Answer: predicted_answer


Average Metric: 5.4 / 2  (270.0):  20%|██        | 2/10 [00:44<02:58, 22.35s/it]

...

Faithful: 5
Detail: 3
Overall: 5
Test Question: Does Weaviate use Hnswlib?
Predicted Answer: predicted_answer


Average Metric: 26.400000000000002 / 10  (264.0): 100%|█| 10/10 [03:41<00:00, 22

Faithful: 5
Detail: 3
Overall: 3
Average Metric: 26.400000000000002 / 10  (264.0%)


,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),The given context does not provide any information about whether there is support for multiple versions of the query/document embedding models to co-exist at a...,3.6
1,How can I retrieve the total object count in a class?,"The given context does not provide any information about how to retrieve the total object count in a class. Therefore, we cannot determine the answer...",1.8
2,How do I get the cosine similarity from Weaviate's certainty?,"The given context does not provide any information about how to get the cosine similarity from Weaviate's certainty. Therefore, we cannot determine the answer based...",1.8
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,The given context does not provide any information about why the quality of search results changes depending on the specified limit or how to fix...,1.8
4,Why did you use GraphQL instead of SPARQL?,The given context does not provide any information about why GraphQL was used instead of SPARQL.,1.6


264.0

# BayesianSignatureOptimizer

In [62]:
from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=llm_metric,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)
third_compiled_rag = teleprompter.compile(RAG(), devset=devset,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Test Question: What is best practice for updating data?
Predicted Answer: predicted_answer


 10%|████▍                                       | 1/10 [00:00<00:03,  2.36it/s]

Faithful: 5
Detail: 3
Overall: 4
Test Question: Can I train my own text2vec-contextionary vectorizer module?
Predicted Answer: predicted_answer


 20%|████████▊                                   | 2/10 [00:00<00:03,  2.65it/s]

...

Test Question: Can I connect my own module?
Predicted Answer: predicted_answer
Faithful: 5
Detail: 5
Overall: 5
Test Question: Can I train my own text2vec-contextionary vectorizer module?
Predicted Answer: predicted_answer


Average Metric: 22.6 / 9  (251.1):  90%|████████ | 9/10 [00:04<00:00,  2.35it/s]

Faithful: 5
Detail: 5
Overall: 5


Average Metric: 24.400000000000002 / 10  (244.0): 100%|█| 10/10 [00:04<00:00,  2
[I 2024-02-11 14:51:18,463] Trial 2 finished with value: 244.0 and parameters: {'10883682544_predictor_instruction': 0, '10883682544_predictor_demos': 0}. Best is trial 0 with value: 290.0.


Test Question: Does Weaviate use Hnswlib?
Predicted Answer: predicted_answer
Faithful: 1
Detail: 2
Overall: 5
Average Metric: 24.400000000000002 / 10  (244.0%)




Answer questions based on the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «There is no performance gain if we have more parallelization than there are available CPUs. However, each thread needs additional memory. So with 32 parallel imports, we had the worst of both worlds: High memory usage and no performance gains beyond 8. With the fix, even if you import from multiple clients, Weaviate automatically handles the parallelization to ensure that it does not exceed the number of CPU cores. As a result, you get the maximum performance without "unnecessary" memory usage. ### HNSW optimization

Next, we optimized memory allocations for the HNSW (vector) i

In [63]:
third_compiled_rag("What do cross encoders do?")

Prediction(
    answer='Cross encoders are ranking models used for content-based re-ranking. They are designed to determine the relevance of a document to a query. Cross encoders take a [query, document] input and output a score indicating the relevance of the document to the query. They can be used to re-rank search results and provide a more personalized and context-aware search experience.'
)

# Check this out!!

Below you can see how the BayesianSignatureOptimizer jointly (1) optimizes the task instruction to:

```
Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.
```

As well as sourcing input-output examples for the prompt!

In [64]:
llm.inspect_history(n=1)





Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Best Answer: ${answer}

---

Context:
[1] «Some models, such as [CLIP](https://openai.com/blog/clip/), are capable of vectorizing multiple data types (images and text in this case) into one vector space, so that an image can be searched by its content using only text. ## Vector embeddings with Weaviate

For this reason, Weaviate is configured to support many different vectorizer models and vectorizer service providers. You can even [bring your own vectors](/developers/weaviate/starter-guides/custom-vectors), for example if you already have a vectorization pipeline available, or if none of th

In [65]:
evaluate(third_compiled_rag, metric=llm_metric)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Test Question: Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions)
Predicted Answer: predicted_answer


Average Metric: 2.0 / 1  (200.0):  10%|█         | 1/10 [00:00<00:02,  3.09it/s]

Faithful: 1
Detail: 3
Overall: 5
Test Question: How can I retrieve the total object count in a class?
Predicted Answer: predicted_answer


Average Metric: 3.8 / 2  (190.0):  20%|██        | 2/10 [00:00<00:02,  2.90it/s]

...

Faithful: 5
Detail: 5
Overall: 5
Test Question: Does Weaviate use Hnswlib?
Predicted Answer: predicted_answer


Average Metric: 29.0 / 10  (290.0): 100%|███████| 10/10 [00:04<00:00,  2.05it/s]

Faithful: 5
Detail: 5
Overall: 5
Average Metric: 29.0 / 10  (290.0%)


,question,answer,llm_metric
0,Is there support to multiple versions of the query/document embedding models to co-exist at a given time? (helps with live experiments of new model versions),No specific information is provided in the given context about support for multiple versions of the query/document embedding models to co-exist at a given time.,2.0
1,How can I retrieve the total object count in a class?,"The given context does not provide specific information on how to retrieve the total object count in a class. However, in Weaviate, you can use...",1.8
2,How do I get the cosine similarity from Weaviate's certainty?,"Weaviate does not directly provide the cosine similarity from its certainty value. The certainty value in Weaviate represents the confidence level of the result, but...",1.6
3,The quality of my search results change depending on the specified limit. Why? How can I fix this?,The quality of search results can change depending on the specified limit because the limit determines the number of results that are returned. When the...,2.0
4,Why did you use GraphQL instead of SPARQL?,The given context does not provide any information about why GraphQL was used instead of SPARQL. The context mainly discusses the performance and functionality of...,4.0


290.0

# Test Set Eval

In [66]:
# Evaluate Uncompiled
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)

In [67]:
evaluate(uncompiled_rag, metric=llm_metric)

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Test Question: Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?
Predicted Answer: predicted_answer


Average Metric: 3.2 / 1  (320.0):   7%|▋         | 1/14 [00:28<06:09, 28.41s/it]

Faithful: 3
Detail: 5
Overall: 5
Test Question: Does Weaviate use pre- or post-filtering ANN index search?
Predicted Answer: predicted_answer


Average Metric: 7.2 / 2  (360.0):  14%|█▍        | 2/14 [00:55<05:31, 27.64s/it]

...

Faithful: 5
Detail: 5
Overall: 5
Test Question: How can I run the latest master branch with Docker Compose?
Predicted Answer: predicted_answer


Average Metric: 37.99999999999999 / 14  (271.4): 100%|█| 14/14 [05:27<00:00, 23.

Faithful: No
Detail: 1
Overall: 5
Error for example in dev set: 		 could not convert string to float: 'No'
Average Metric: 37.99999999999999 / 14  (271.4%)


,question,answer,llm_metric
0,Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?,"No, not all ANN algorithms are potential candidates to become an indexation plugin in Weaviate. Reasoning: The context mentions that Weaviate has implemented the HNSW...",3.2
1,Does Weaviate use pre- or post-filtering ANN index search?,"Weaviate uses pre-filtering ANN index search. Reasoning: In the context, it is mentioned that Weaviate uses the HNSW indexing algorithm, which builds a hierarchical representation...",4.0
2,How does Weaviate's vector and scalar filtering work?,nan,0.0
3,"What would you say is more important for query speed in Weaviate: More CPU power, or more RAM?",More RAM is more important for query speed in Weaviate.,1.2
4,"Data import takes long / is slow, what is causing this and what can I do?",The slow data import in Weaviate could be caused by the large number of data objects being imported and the need to maintain data integrity....,3.8


271.43

In [68]:
evaluate(compiled_rag, metric=llm_metric)

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Test Question: Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?
Predicted Answer: predicted_answer


Average Metric: 3.6 / 1  (360.0):   7%|▋         | 1/14 [00:28<06:07, 28.25s/it]

Faithful: 4
Detail: 5
Overall: 5
Test Question: Does Weaviate use pre- or post-filtering ANN index search?
Predicted Answer: predicted_answer


Average Metric: 7.6 / 2  (380.0):  14%|█▍        | 2/14 [00:54<05:24, 27.04s/it]

...

Faithful: 1
Detail: 2
Overall: 5
Test Question: How can I run the latest master branch with Docker Compose?
Predicted Answer: predicted_answer


Average Metric: 40.0 / 14  (285.7): 100%|███████| 14/14 [05:47<00:00, 24.83s/it]

Faithful: 1
Detail: 1
Overall: 5
Average Metric: 40.0 / 14  (285.7%)


,question,answer,llm_metric
0,Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?,"No, not all ANN algorithms are potential candidates to become an indexation plugin in Weaviate. According to the given context, Weaviate has implemented the HNSW...",3.6
1,Does Weaviate use pre- or post-filtering ANN index search?,"Weaviate uses pre-filtering ANN index search. According to the given context, Weaviate uses the HNSW indexing algorithm, which builds a hierarchical representation of the index...",4.0
2,How does Weaviate's vector and scalar filtering work?,The given context does not provide any information about how Weaviate's vector and scalar filtering works.,1.6
3,"What would you say is more important for query speed in Weaviate: More CPU power, or more RAM?","According to the given context, increasing the frequency of garbage collector (GC) cycles will use more CPU, while increasing RAM will incur additional costs. It...",2.4
4,"Data import takes long / is slow, what is causing this and what can I do?",The cause of slow data import in Weaviate could be the large volume of data being imported. Weaviate recommends using the batch import feature for...,3.4


285.71

In [69]:
evaluate(second_compiled_rag, metric=llm_metric)

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Test Question: Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?
Predicted Answer: predicted_answer


Average Metric: 3.6 / 1  (360.0):   7%|▋         | 1/14 [00:22<04:47, 22.08s/it]

Faithful: 5
Detail: 3
Overall: 5
Test Question: Does Weaviate use pre- or post-filtering ANN index search?
Predicted Answer: predicted_answer


Average Metric: 5.4 / 2  (270.0):  14%|█▍        | 2/14 [00:55<05:41, 28.48s/it]

...

Faithful: 5
Detail: 3
Overall: 5
Test Question: How can I run the latest master branch with Docker Compose?
Predicted Answer: predicted_answer


Average Metric: 35.8 / 14  (255.7): 100%|███████| 14/14 [05:51<00:00, 25.14s/it]

Faithful: 1
Detail: 1
Overall: 5
Average Metric: 35.8 / 14  (255.7%)


,question,answer,llm_metric
0,Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?,"The given context does not provide any information about whether all ANN algorithms are potential candidates to become an indexation plugin in Weaviate. Therefore, we...",3.6
1,Does Weaviate use pre- or post-filtering ANN index search?,The given context does not provide information about whether Weaviate uses pre- or post-filtering ANN index search.,1.8
2,How does Weaviate's vector and scalar filtering work?,"The given context does not provide any information about how Weaviate's vector and scalar filtering work. Therefore, we cannot determine the answer based on the...",1.8
3,"What would you say is more important for query speed in Weaviate: More CPU power, or more RAM?","Based on the given information, it is not explicitly stated whether more CPU power or more RAM is more important for query speed in Weaviate....",4.0
4,"Data import takes long / is slow, what is causing this and what can I do?","The given context does not provide specific information about the causes of slow data import in Weaviate or the possible solutions. Therefore, we cannot determine...",0.8


255.71

In [70]:
evaluate(third_compiled_rag, metric=llm_metric)

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Test Question: Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?
Predicted Answer: predicted_answer


Average Metric: 3.2 / 1  (320.0):   7%|▋         | 1/14 [00:34<07:25, 34.28s/it]

Faithful: 3
Detail: 5
Overall: 5
Test Question: Does Weaviate use pre- or post-filtering ANN index search?
Predicted Answer: predicted_answer


Average Metric: 6.0 / 2  (300.0):  14%|█▍        | 2/14 [00:59<05:45, 28.75s/it]

...

Faithful: 5
Detail: 5
Overall: 5
Test Question: How can I run the latest master branch with Docker Compose?
Predicted Answer: predicted_answer


Average Metric: 48.4 / 14  (345.7): 100%|███████| 14/14 [05:55<00:00, 25.37s/it]

Faithful: 1
Detail: 5
Overall: 1
Average Metric: 48.4 / 14  (345.7%)


,question,answer,llm_metric
0,Are all ANN algorithms potential candidates to become an indexation plugin in Weaviate?,"No, not all ANN (Approximate Nearest Neighbor) algorithms are potential candidates to become an indexation plugin in Weaviate. The context mentions that Weaviate has implemented...",3.2
1,Does Weaviate use pre- or post-filtering ANN index search?,"Weaviate uses pre-filtering ANN index search. The context mentions that Weaviate implements the HNSW indexing algorithm, which builds a hierarchical representation of the index in...",2.8
2,How does Weaviate's vector and scalar filtering work?,The given context does not provide specific information about how Weaviate's vector and scalar filtering works. It mentions that Weaviate looks for approximate (close enough)...,3.4
3,"What would you say is more important for query speed in Weaviate: More CPU power, or more RAM?","Based on the information provided in the context, it is more important to have more RAM for query speed in Weaviate. The context mentions optimizations...",4.0
4,"Data import takes long / is slow, what is causing this and what can I do?","The context mentions that Weaviate offers a batch import feature for adding data objects in bulk, which is recommended for speeding up the import process....",4.0


345.71